In [1]:
import glob, os, json
import subprocess
import pandas as pd
import shutil

# Build lookup table

In [31]:
transl_path = "/home/craig.car/repos/chiron/align_texts_project/data/translations_repositories.csv"

In [32]:
transl_df = pd.read_csv(transl_path)
transl_df = transl_df.rename(columns={"CTS-URN/s":"cts-urns_grp", "Original File Name/Location":"Original_File_Name", 
                                      "Addiional File location/locations":"Additional_File_locations", "Translation Language":"translation_language",
                                     "New File Name/s":"New_File_Name"})

In [33]:
transl_df.loc[transl_df["Editor; Translator"] == "Hickie, William James"]

,Author Name,WORK IDs,cts-urns_grp,# of translations,Edition Title,Editor; Translator,Publisher,Date,Original_File_Name,Additional_File_locations,Source Edition LInk,translation_language,Worldcat Link,Work/s Split Up/Renamed in Branch,New_File_Name,Notes,TOC,Additional TOC
31,Aristophanes,0019.001 0019.002 0019.003 0019.004 0019.005 0...,urn:cts:greekLit:tlg0019.tlg001.ogl-eng1 urn:c...,11.0,"The Comedies of Aristophanes, Vol 1: Archarnia...","Hickie, William James",Bohn,1858,aristophanes_1_1858.xml,aristophanes_1907_1.xml,https://archive.org/details/cu31924087931675 (...,English,NaN,no,NaN,"Second file misname, it is volume 2 of this ed...",NaN,NaN
32,Aristophanes,NaN,NaN,5.0,"The Comedies of Aristophanes, Vol 2 (DUPLICATE)","Hickie, William James",Bohn,1853,aristophanes_2_1853.xml,NaN,https://archive.org/details/cu31924087931683,English,NaN,no,NaN,NaN,NaN,NaN
41,Aristophanes; Plautus,"0019.011, 119.10",urn:cts:greekLit:tlg0019.tlg011.ogl-eng3 urn:c...,2.0,"The Comedies of Aristophanes, Vol 6: The Plut...","Hickie, William James",Bell,1907,aristophanes_1907_2.xml,NaN,https://archive.org/details/ComediesOfAristoph...,English,NaN,no,NaN,Work contains a stealth translation of a work ...,NaN,NaN


In [34]:
transl_df.loc[0,"cts-urns_grp"]

'urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:cts:greekLit:tlg0085.tlg002.ogl-eng1 urn:cts:greekLit:tlg0085.tlg003.ogl-eng1 urn:cts:greekLit:tlg0085.tlg004.ogl-eng1 urn:cts:greekLit:tlg0085.tlg005.ogl-eng1 urn:cts:greekLit:tlg0085.tlg006.ogl-eng1 urn:cts:greekLit:tlg0085.tlg007.ogl-eng1'

In [35]:
transl_df.loc[0,"cts-urns_grp"].split()

['urn:cts:greekLit:tlg0085.tlg001.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg002.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg003.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg004.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg005.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg006.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg007.ogl-eng1']

In [36]:
transl_df["cts-urns_uniques"] = transl_df["cts-urns_grp"].str.split()

In [37]:
transl_df = transl_df.explode("cts-urns_uniques", ignore_index=True)

In [38]:
transl_df["ctsurns"] = transl_df["cts-urns_uniques"].str.rsplit(".",1).str[0]

In [39]:
transl_df["filename1"] = transl_df["Original_File_Name"].str.split(".").str[0]

In [40]:
transl_df["filename2"] = transl_df["Additional_File_locations"].str.split(".").str[0]

In [41]:
transl_df.loc[transl_df["ctsurns"]=="urn:cts:greekLit:tlg0640.tlg001","New_File_Name"]

55    tlg0640.tlg001.ogl-eng1.xml
Name: New_File_Name, dtype: object

In [42]:
# don't split filenames in this col
transl_df["filename3"] = transl_df["New_File_Name"]

In [43]:
transl_df.head()

,Author Name,WORK IDs,cts-urns_grp,# of translations,Edition Title,Editor; Translator,Publisher,Date,Original_File_Name,Additional_File_locations,...,Work/s Split Up/Renamed in Branch,New_File_Name,Notes,TOC,Additional TOC,cts-urns_uniques,ctsurns,filename1,filename2,filename3
0,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1,urn:cts:greekLit:tlg0085.tlg001,aeschylus_1886,NaN,NaN
1,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg002.ogl-eng1,urn:cts:greekLit:tlg0085.tlg002,aeschylus_1886,NaN,NaN
2,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg003.ogl-eng1,urn:cts:greekLit:tlg0085.tlg003,aeschylus_1886,NaN,NaN
3,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg004.ogl-eng1,urn:cts:greekLit:tlg0085.tlg004,aeschylus_1886,NaN,NaN
4,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg005.ogl-eng1,urn:cts:greekLit:tlg0085.tlg005,aeschylus_1886,NaN,NaN


In [44]:
transl_df.isna().sum()

Author Name                            25
WORK IDs                               15
cts-urns_grp                           19
# of translations                       5
Edition Title                           0
Editor; Translator                      1
Publisher                               2
Date                                    4
Original_File_Name                     12
Additional_File_locations            1074
Source Edition LInk                    14
translation_language                   22
Worldcat Link                        1292
Work/s Split Up/Renamed in Branch    1532
New_File_Name                        1597
Notes                                1239
TOC                                  1252
Additional TOC                       1547
cts-urns_uniques                       19
ctsurns                                19
filename1                              12
filename2                            1074
filename3                            1597
dtype: int64

In [45]:
transl_df.fillna("", inplace=True)

In [46]:
transl_df["ctsurns"].duplicated().sum()

796

In [47]:
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001", "filename2"]

485    euripides_2_1884
493                    
494                    
500                    
Name: filename2, dtype: object

In [48]:
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001", "filename2"]

485    euripides_2_1884
493                    
494                    
500                    
Name: filename2, dtype: object

In [49]:
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0085.tlg001", "filename2"]

0     
7     
14    
24    
34    
Name: filename2, dtype: object

In [50]:
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0006.tlg001"]

,Author Name,WORK IDs,cts-urns_grp,# of translations,Edition Title,Editor; Translator,Publisher,Date,Original_File_Name,Additional_File_locations,...,Work/s Split Up/Renamed in Branch,New_File_Name,Notes,TOC,Additional TOC,cts-urns_uniques,ctsurns,filename1,filename2,filename3
485,Euripides,0006.007 0006.015 0006.003 0006.005 0006.002 0...,urn:cts:greekLit:tlg0006.tlg007.ogl-fre1 urn:c...,18.0,"Euripide, Volumes 1-2. 1, Hékabé. Les Phoini...","Leconte de Lisle, Charles-Marie, 1818-1894\r\n...",Lemerre,1884,euripides_1_1884.xml,euripides_2_1884.xml,...,,,,,,urn:cts:greekLit:tlg0006.tlg001.ogl-fre1,urn:cts:greekLit:tlg0006.tlg001,euripides_1_1884,euripides_2_1884,
493,Euripides,0006.015 0006.001,urn:cts:greekLit:tlg0006.tlg015.ogl-ger1 urn:c...,2.0,Die Dramen des Euripides: Die Phönizierinnen (...,"Minckwitz, Johannes, 1812-1885\r\n ...",Hoffman,1857,euripides_1857_1,,...,,,Line 6219-starts a second Volume bound in this...,,,urn:cts:greekLit:tlg0006.tlg001.ogl-ger1,urn:cts:greekLit:tlg0006.tlg001,euripides_1857_1,,
494,Euripides,0006.001 0006.002 0006.003 0006.011,urn:cts:greekLit:tlg0006.tlg001.ogl-ger2 urn:c...,4.0,"Griechische Tragödien, Volume 3. Euripides: D...","Wilamowitz-Moellendorff, Ulrich von 1848-1931",Weidmann,1906,euripides_1906_3,,...,,,,,,urn:cts:greekLit:tlg0006.tlg001.ogl-ger2,urn:cts:greekLit:tlg0006.tlg001,euripides_1906_3,,
500,Euripides,0006.001,urn:cts:greekLit:tlg0006.tlg001.ogl-ita1,1.0,Il Ciclope,"Romagnoli, Ettore, 1871-1938",Casa Editrice Italiana di A. Quattrin,1911,euripides_1911.xml,,...,,,,,,urn:cts:greekLit:tlg0006.tlg001.ogl-ita1,urn:cts:greekLit:tlg0006.tlg001,euripides_1911,,


In [51]:
# lookup_table = {}
# keys = transl_df["ctsurns"].unique()
# for ctsurn in keys:
#     # print(ctsurn)
#     # lookup_table[ctsurn] = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename"])
#     filenames = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename1"])
#     # print(vals)
#     lang = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "translation_language"])
#     # print(lang)
#     # print(lang.index)
   
#     # print(test)
#     for other_name in list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename2"]):
#         if other_name != "":
#             filenames.append(other_name)
#             # get lang
#             language = transl_df.loc[transl_df["filename2"] == other_name, "translation_language"]
#             # print(other_name)
#             # print(language)
#             break
            
    
#     # for other_name in list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename2"]):
#     #     # don't add empty string values (previously NaN)
#     #     if other_name != "":
#     #         vals.append(other_name)
#     #         print(type(other_name))
#     #         break
#     # vals = list(zip(filenames,lang))
#     # # turn back to list to write to json
#     # lookup_table[ctsurn] = list(vals)

In [52]:
lookup_table = {}
keys = transl_df["ctsurns"].unique()
for ctsurn in keys:
    # lookup_table[ctsurn] = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename"])
    vals_initial = set(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename1"])
    vals = set([name for name in vals_initial if name != ""])
    # should only print "urn:cts:greekLit:tlg0001.tlg001" because I deleted the filename in "filename1" col
    if vals_initial != vals:
        print(ctsurn)
    for other_files in list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename2"]):
        # don't add empty string values (previously NaN)
        if other_files != "":
            new_files = other_files.split()
            for file in new_files:
                vals.add(file)
    for other_files in list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename3"]):
        # don't add empty string values (previously NaN)
        if other_files != "":
            new_files = other_files.split()
            for file in new_files:
                vals.add(file)
    # turn back to list to write to json
    lookup_table[ctsurn] = list(vals)

urn:cts:greekLit:tlg0019.tlg003
urn:cts:greekLit:tlg7000.tlg001
urn:cts:greekLit:tlg0001.tlg001
urn:cts:greekLit:tlg0019.tlg004
urn:cts:greekLit:tlg0019.tlg010
urn:cts:greekLit:tlg0019.tlg005
urn:cts:greekLit:tlg0019.tlg009
urn:cts:greekLit:tlg0006.tlg018
urn:cts:greekLit:tlg0016.tlg001
urn:cts:greekLit:tlg0012.tlg002
urn:cts:greekLit:tlg0099.tlg001


In [53]:
# spot check dict values
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0085.tlg004", "filename1"]

3     aeschylus_1886
10    aeschylus_1888
17    aeschylus_1909
27    aeschylus_1872
37    aeschylus_1868
Name: filename1, dtype: object

In [54]:
lookup_table["urn:cts:greekLit:tlg0011.tlg004"]

['sophocles_1811',
 'sophocles_1804',
 'greekDramas_1912',
 'sophocles_1907',
 'sophocles_1873_1',
 'sophocles_1905_en']

In [55]:
lookup_table["urn:cts:greekLit:tlg0074.tlg001"]

['arrian_1_1802', 'arrian_1862', 'arrian_1802(3)']

In [56]:
len("/scratch/craig.car/src_data/")

28

In [57]:
"/scratch/craig.car/src_data/urn:cts:greekLit:tlg0001.tlg001"[28:]

'urn:cts:greekLit:tlg0001.tlg001'

In [59]:
# path_out = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"
# with open(path_out, 'w') as fp:
#     json.dump(lookup_table, fp)

# Load lookup table to test it worked

In [53]:
# test lookup table
lookup_path = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"

with open(lookup_path) as f:
    lookup_test = json.load(f)


In [54]:
lookup_test["urn:cts:greekLit:tlg0551.tlg002"]

['appian_1832_10', 'Appianus_1828', 'appian_1830_5']

In [61]:
lookup_test["urn:cts:greekLit:tlg0001.tlg001"]

['apollonius_1889',
 'tlg0001.tlg001.opp-eng1.xml',
 'apolloniusRhodius_1_1791',
 'apolloniusRhodius_1892']

In [56]:
lookup_test == lookup_table

True

# Run Vecalign - moved to python file

In [46]:
params = {
    'QUEUE':'short',
    'SWORKERS':2,
    'SMEM':4
}

labse_vec_cpu = 'srun --partition=short --nodes=1 --mem=4GB --time=01:00:00 /bin/bash -c "source /home/craig.car/miniconda3/bin/activate; conda activate labse_vec_pipeline; {command};"'

src_dir = "/scratch/craig.car/src_data/"
fr_dir = "/scratch/craig.car/french_trans-dev/"
en_dir = "/scratch/craig.car/english_trans-dev/"
it_dir_sub = "/scratch/craig.car/italian_trans-dev/as_yet_unedited_DDD/"
it_dir = "/scratch/craig.car/italian_trans-dev/"
de_dir = "/scratch/craig.car/german_trans-dev/"

rslts_dir = "/home/craig.car/repos/chiron/chironata/data/alignments_rslts/"

In [48]:
for src_emb in glob.iglob(src_dir+"*.emb"):
    ctsurn = os.path.splitext(os.path.basename(src_emb))[0]
    src_prefix = src_dir+ctsurn
    tgt_files = lookup_table[ctsurn]
    for tgt_name in tgt_files:
        tgt_prefix = fr_dir+tgt_name
        # check if a fr translation exists
        if os.path.isfile(tgt_prefix+".emb") != False: 
            params['command'] = f'./vecalign.py --alignment_max_size 8 --src {src_prefix+".sents"} --tgt {tgt_prefix+".sents"} --src_embed {src_prefix+".overlaps"} {src_emb} --tgt_embed {tgt_prefix+".overlaps"} {tgt_prefix+".emb"}'
            # print(labse_vec_cpu.format(**params))
            if os.path.isfile(f"{rslts_dir}{ctsurn}_{tgt_name}.rslts") == False:
                print(f"working on {ctsurn}_{tgt_name}")
                subprocess.run(labse_vec_cpu.format(**params),shell=True)
    
            

## For English files

In [32]:
for src_emb in glob.iglob(src_dir+"*de_dir"):
    ctsurn = os.path.splitext(os.path.basename(src_emb))[0]
    src_prefix = src_dir+ctsurn
    
    # some en files are in partial cts form: author.text code only
    ctsurn_auth_txt = os.path.splitext(os.path.basename(src_emb))[0].split(":")[3]
    en_cts = en_dir+ctsurn_auth_txt
    # check if such a file exists in en_dir
    files = os.listdir(en_dir)
    for file in files:
        if file.startswith(ctsurn_auth_txt) and file.endswith(".emb"):
            en_cts_emb = en_dir+file
            # get prefix
            file_prefix = os.path.splitext(os.path.basename(en_cts_emb))[0]
            tgt_prefix = en_dir + file_prefix
            params['command'] = f'./vecalign.py --alignment_max_size 8 --src {src_prefix+".sents"} --tgt {tgt_prefix+".sents"} --src_embed {src_prefix+".overlaps"} {src_emb} --tgt_embed {tgt_prefix+".overlaps"} {tgt_prefix+".emb"}'
            # print(labse_vec_cpu.format(**params))
            # check if file exists already
            if os.path.isfile(f"{rslts_dir}{ctsurn}_{file_prefix}.rslts") == False:
                print(f"working on {ctsurn}_{file_prefix}")
                subprocess.run(labse_vec_cpu.format(**params),shell=True)
            
    # check for en translations under spreadsheet name
    tgt_files = lookup_table[ctsurn]
    for tgt_name in tgt_files:
        tgt_prefix = en_dir+tgt_name
        # check if an en translation exists
        if os.path.isfile(tgt_prefix+".emb") != False:
            params['command'] = f'./vecalign.py --alignment_max_size 8 --src {src_prefix+".sents"} --tgt {tgt_prefix+".sents"} --src_embed {src_prefix+".overlaps"} {src_emb} --tgt_embed {tgt_prefix+".overlaps"} {tgt_prefix+".emb"}'
            # check if file exists already
            if os.path.isfile(f"{rslts_dir}{ctsurn}_{tgt_name}.rslts") == False:
                print(f"working on {ctsurn}_{tgt_name}")
                # print(labse_vec_cpu.format(**params))
                subprocess.run(labse_vec_cpu.format(**params),shell=True)
         
    

working on urn:cts:greekLit:tlg0007.tlg052_plutarch_4_1908


2023-11-29 10:19:48,896  WARNI  expected an embedding size of 1024, got 768
2023-11-29 10:19:49,705  WARNI  expected an embedding size of 1024, got 768


/home/craig.car/repos/chiron/chironata/data/alignments_rslts/urn:cts:greekLit:tlg0007.tlg052_plutarch_4_1908
working on urn:cts:greekLit:tlg0007.tlg052_plutarch_3_1908


2023-11-29 10:20:01,922  WARNI  expected an embedding size of 1024, got 768
2023-11-29 10:20:02,447  WARNI  expected an embedding size of 1024, got 768


/home/craig.car/repos/chiron/chironata/data/alignments_rslts/urn:cts:greekLit:tlg0007.tlg052_plutarch_3_1908
working on urn:cts:greekLit:tlg0007.tlg053_plutarch_4_1908


...

/home/craig.car/repos/chiron/chironata/data/alignments_rslts/urn:cts:latinLit:stoa0255.stoa012_seneca_1889
working on urn:cts:latinLit:stoa0255.stoa013_seneca_1889


2023-11-29 11:18:52,950  WARNI  expected an embedding size of 1024, got 768
2023-11-29 11:18:53,245  WARNI  expected an embedding size of 1024, got 768


/home/craig.car/repos/chiron/chironata/data/alignments_rslts/urn:cts:latinLit:stoa0255.stoa013_seneca_1889
working on urn:cts:latinLit:stoa0255.stoa014_seneca_1889


2023-11-29 11:19:00,279  WARNI  expected an embedding size of 1024, got 768
2023-11-29 11:19:00,603  WARNI  expected an embedding size of 1024, got 768


/home/craig.car/repos/chiron/chironata/data/alignments_rslts/urn:cts:latinLit:stoa0255.stoa014_seneca_1889


## Italian files

In [96]:
for src_emb in glob.iglob(src_dir+"*.emb"):
    ctsurn = os.path.splitext(os.path.basename(src_emb))[0]
    src_prefix = src_dir+ctsurn
    tgt_files = lookup_table[ctsurn]
    for tgt_name in tgt_files:
        tgt_prefix = it_dir+tgt_name
        # print(tgt_prefix)
        # check if translation exists
        # print(tgt_prefix+".emb")
        if os.path.isfile(tgt_prefix+".emb") != False:
            print(tgt_prefix+".emb")
            params['command'] = f'./vecalign.py --alignment_max_size 8 --src {src_prefix+".sents"} --tgt {tgt_prefix+".sents"} --src_embed {src_prefix+".overlaps"} {src_emb} --tgt_embed {tgt_prefix+".overlaps"} {tgt_prefix+".emb"}'
            print(f"{rslts_dir}{ctsurn}_{tgt_name}.rslts")
            if os.path.isfile(f"{rslts_dir}{ctsurn}_{tgt_name}.rslts") == False:
                print(f"working on {ctsurn}_{tgt_name}")
                subprocess.run(labse_vec_cpu.format(**params),shell=True)

/scratch/craig.car/italian_trans-dev/xenophon_1863.emb
/home/craig.car/repos/chiron/chironata/data/alignments_rslts/urn:cts:greekLit:tlg0641.tlg001_xenophon_1863.rslts


## German files

In [61]:
### move xml files from subdirectories to parent file ###
# for subdir, dirs, files in os.walk(de_dir):
#     for file in files:
#         if file.endswith((".xml")):
#             print (os.path.join(subdir, file))
#             new_path = "/scratch/craig.car/german_trans-dev/"
#             if os.path.isfile(new_path+file) == False:
#                 shutil.move((os.path.join(subdir, file)), new_path)

In [62]:
# for src_emb in glob.iglob(src_dir+"*.emb"):
#     ctsurn = os.path.splitext(os.path.basename(src_emb))[0]
#     src_prefix = src_dir+ctsurn
#     tgt_files = lookup_table[ctsurn]
#     for tgt_name in tgt_files:
#         tgt_prefix = de_dir+tgt_name
#         # print(tgt_prefix)
#         # check if translation exists
#         # print(tgt_prefix+".emb")
#         if os.path.isfile(tgt_prefix+".emb") != False:
#             # print(tgt_prefix+".emb")
#             params['command'] = f'./vecalign.py --alignment_max_size 8 --src {src_prefix+".sents"} --tgt {tgt_prefix+".sents"} --src_embed {src_prefix+".overlaps"} {src_emb} --tgt_embed {tgt_prefix+".overlaps"} {tgt_prefix+".emb"}'
#             if os.path.isfile(f"{rslts_dir}{ctsurn}_{tgt_name}.rslts") == False:
#                 print(f"working on {ctsurn}_{tgt_name}")
#                 subprocess.run(labse_vec_cpu.format(**params),shell=True)

In [46]:
en_eg = "/scratch/craig.car/english_trans-dev/tlg0019.tlg010.ogl-eng2.xml"
os.path.splitext(os.path.splitext(os.path.basename(en_eg))[0])[0]

'tlg0019.tlg010'

In [1]:
import markdown as md

In [13]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [1, 2, 3]}, index=['a', 'a', 'b'])
print(df.to_markdown(buf="./README.md"))

None


## Test using 2 Th fr translations

In [27]:
import pandas as pd
from IPython.display import display, HTML

'''
https://stackoverflow.com/questions/40990700/pandas-dataframes-in-jupyter-columns-of-equal-width-and-centered
https://stackoverflow.com/questions/34322448/pretty-printing-newlines-inside-a-string-in-a-pandas-dataframe
'''

data = [
       {'Greek': "ὃς δ᾽ ἂν ὑμῶν παραμείνῃ, ὁρῶν ὃν τρόπον ἡμεῖς τάς τε δίκας δικάζομεν καὶ τἆλλα τὴν πόλιν διοικοῦμεν, ἤδη φαμὲν τοῦτον ὡμολογηκέναι ἔργῳ ἡμῖν ἃ ἂν ἡμεῖς κελεύωμεν ποιήσειν ταῦτα, καὶ τὸν μὴ πειθόμενον τριχῇ φαμεν ἀδικεῖν, ὅτι τε γεννηταῖς οὖσιν ἡμῖν οὐ πείθεται, καὶ ὅτι τροφεῦσι, καὶ ὅτι ὁμολογήσας ἡμῖν πείσεσθαι οὔτε πείθεται οὔτε πείθει ἡμᾶς, εἰ μὴ καλῶς τι ποιοῦμεν, προτιθέντων ἡμῶν καὶ οὐκ ἀγρίως ἐπιταττόντων ποιεῖν ἃ ἂν κελεύωμεν, ἀλλὰ ἐφιέντων δυοῖν θάτερα, ἢ πείθειν ἡμᾶς ἢ ποιεῖν, τούτων οὐδέτερα ποιεῖ.", 'Fowler': '[0] But we say that whoever of you stays here, seeing how we administer justice and how we govern the state in other respects, has thereby entered into an agreement with us to do what we command;\n[1] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”', 'Jowett': '[0] But he who has experience of the manner in which we order justice and administer the State, and still remains, has entered into an implied contract that he will do as we command him.\n[1] And he who disobeys us is, as we maintain, thrice wrong:\n[2] first, because in disobeying us he is disobeying his parents;\n[3] secondly, because we are the authors of his education;\n[4] thirdly, because he has made an agreement with us that he will duly obey our commands;\n[5] and he neither obeys them nor convinces us that our commands are wrong;\n[6] and we do not rudely impose them, but give him the alternative of obeying or convincing us;'},
    {'Greek': 'ταύταις δή φαμεν καὶ σέ, ὦ Σώκρατες, ταῖς αἰτίαις ἐνέξεσθαι, εἴπερ ποιήσεις ἃ ἐπινοεῖς, καὶ οὐχ ἥκιστα Ἀθηναίων σέ, ἀλλ᾽ ἐν τοῖς μάλιστα.', 'Fowler':'[2] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”', 'Jowett':'[7] that is what we offer and he does neither.\n[8] These are the sort of accusations to which, as we were saying, you, Socrates, will be exposed if you accomplish your intentions;\n[9] you, above all other Athenians."'}
       ]
df = pd.DataFrame(data)

d = dict(selector="th",
    props=[('text-align', 'center')])
df.style.set_properties(**{'width':'30em', 'text-align':'left', 'white-space': 'pre-wrap'})\
        .set_table_styles([d])

,Greek,Fowler,Jowett
0,"ὃς δ᾽ ἂν ὑμῶν παραμείνῃ, ὁρῶν ὃν τρόπον ἡμεῖς τάς τε δίκας δικάζομεν καὶ τἆλλα τὴν πόλιν διοικοῦμεν, ἤδη φαμὲν τοῦτον ὡμολογηκέναι ἔργῳ ἡμῖν ἃ ἂν ἡμεῖς κελεύωμεν ποιήσειν ταῦτα, καὶ τὸν μὴ πειθόμενον τριχῇ φαμεν ἀδικεῖν, ὅτι τε γεννηταῖς οὖσιν ἡμῖν οὐ πείθεται, καὶ ὅτι τροφεῦσι, καὶ ὅτι ὁμολογήσας ἡμῖν πείσεσθαι οὔτε πείθεται οὔτε πείθει ἡμᾶς, εἰ μὴ καλῶς τι ποιοῦμεν, προτιθέντων ἡμῶν καὶ οὐκ ἀγρίως ἐπιταττόντων ποιεῖν ἃ ἂν κελεύωμεν, ἀλλὰ ἐφιέντων δυοῖν θάτερα, ἢ πείθειν ἡμᾶς ἢ ποιεῖν, τούτων οὐδέτερα ποιεῖ.","[0] But we say that whoever of you stays here, seeing how we administer justice and how we govern the state in other respects, has thereby entered into an agreement with us to do what we command; [1] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”","[0] But he who has experience of the manner in which we order justice and administer the State, and still remains, has entered into an implied contract that he will do as we command him. [1] And he who disobeys us is, as we maintain, thrice wrong: [2] first, because in disobeying us he is disobeying his parents; [3] secondly, because we are the authors of his education; [4] thirdly, because he has made an agreement with us that he will duly obey our commands; [5] and he neither obeys them nor convinces us that our commands are wrong; [6] and we do not rudely impose them, but give him the alternative of obeying or convincing us;"
1,"ταύταις δή φαμεν καὶ σέ, ὦ Σώκρατες, ταῖς αἰτίαις ἐνέξεσθαι, εἴπερ ποιήσεις ἃ ἐπινοεῖς, καὶ οὐχ ἥκιστα Ἀθηναίων σέ, ἀλλ᾽ ἐν τοῖς μάλιστα.","[2] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”","[7] that is what we offer and he does neither. [8] These are the sort of accusations to which, as we were saying, you, Socrates, will be exposed if you accomplish your intentions; [9] you, above all other Athenians."""


In [3]:
from ast import literal_eval
#### vecalign function ####
def read_alignments(fin):
    alignments = []
    with open(fin, 'rt', encoding="utf-8") as infile:
        for line in infile:
            fields = [x.strip() for x in line.split(':') if len(x.strip())]
            if len(fields) < 2:
                raise Exception('Got line "%s", which does not have at least two ":" separated fields' % line.strip())
            try:
                src = literal_eval(fields[0])
                tgt = literal_eval(fields[1])
            except:
                raise Exception('Failed to parse line "%s"' % line.strip())
            alignments.append((src, tgt))

    # I know bluealign files have a few entries entries missing,
    #   but I don't fix them in order to be consistent previous reported scores
    return alignments

In [4]:
from functions import load_txt_as_lst

In [5]:
th1_rslts = read_alignments("/home/craig.car/repos/chiron/chironata/data/alignments_rslts/thucydides_1863.rslts")
th2_rslts = read_alignments("/home/craig.car/repos/chiron/chironata/data/alignments_rslts/thucydides_1_1852.rslts")
el_sents = load_txt_as_lst("/home/craig.car/repos/chiron/chironata/data/src_data/urn:cts:greekLit:tlg0003.tlg001.sents")
th1_sents = load_txt_as_lst("/home/craig.car/repos/chiron/chironata/data/french_trans-dev/thucydides_1863.sents")

In [6]:
th1_rslts[0:20]

[([], [0]),
 ([], [1]),
 ([], [2]),
 ([], [3]),
 ([], [4]),
 ([], [5]),
 ([0], [6, 7, 8, 9, 10, 11, 12]),
 ([], [13]),
 ([], [14]),
 ([1], [15, 16, 17, 18]),
 ([], [19]),
 ([], [20]),
 ([], [21]),
 ([], [22]),
 ([], [23]),
 ([], [24]),
 ([], [25]),
 ([], [26]),
 ([], [27]),
 ([], [28])]

In [7]:
th1_sents[0]

'Introduction.\n'

In [8]:
el_series = []
for alignment in th1_rslts:
    src_pred = alignment[0]
    src_cell = ""
    if src_pred == []:
        el_series.append("")
    else:
        for src_idx in src_pred:
            src_sent = el_sents[src_idx]
            src_cell += f"[{src_idx}] {src_sent}\n"
        el_series.append(src_cell)
        
        # for src_idx in src_pred:
        #     el_series.append(el_sents[src_idx])

In [9]:
len(el_series)

4678

In [12]:
th_df = pd.DataFrame(el_series)
th_df.columns=["Greek"]
th_df.head(10)

,Greek
0,
1,
2,
3,
4,
5,
6,[0] Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον ...
7,
8,
9,[1] κίνησις γὰρ αὕτη μεγίστη δὴ τοῖς Ἕλλησιν ἐ...


In [13]:
th1_series = []
for alignment in th1_rslts:
    tgt_pred = alignment[1]
    tgt_cell = ""
    if tgt_pred == []:
        th1_series.append("")
    else:
        for tgt_idx in tgt_pred:
            tgt_sent = th1_sents[tgt_idx]
            tgt_cell += f"[{tgt_idx}] {tgt_sent}\n"
    th1_series.append(tgt_cell)

In [14]:
th1_series[:20]

['[0] Introduction.\n\n',
 '[1] L’auteur passe en revue les temps primitifs de la Grèce, afin de prouver que la guerre du Péloponèse a surpassé en importance toutes les guerres qui avaient précédé, chap.\n\n',
 '[2] i-xix.\n\n',
 '[3] But qu’il s’est proposé dans la rédaction de son ouvrage;\n\n',
 '[4] méthode et moyens qu’il a employés pour y parvenir, chap.\n\n',
 '[5] xx-xxiii.\n\n',
 '[6] — Épidamne et Potidée.\n\n[7] Événements qui provoquèrent la guerre du Péloponèse.\n\n[8] Affaire d’Épidamne·, guerre entre Corcyre et Corinthe;\n\n[9] premier combat naval, chap.\n\n[10] xxiv-xxxi.\n\n[11] Les Gorcyréens obtiennent l’alliance d’Athènes.\n\n[12] Discours des Corcyréens et des Corinthiens, chap. xxxiixliii.\n\n',
 '[13] Second combat naval entre les Corcyréens et les Corinthiens;\n\n',
 '[14] fin de la guerre de Corcyre, chap.\n\n',
 '[15] xliv-lv.\n\n[16] Défection de Potidée;\n\n[17] combat livré sous les murs de cette ville et siége commencé par les Athéniens, chap.\n\n[18] lvi

In [15]:
th1_series[-1]

'[10353] Il se rendit en premier lieu à Éphèse, où il offrit un sacrifice à Diane.Quand l’hiver qui suit cet été aura pris fin, la vingt et unième année de la guerre sera terminée2.\n\n'

In [16]:
el_series[-1]

'[4416] καὶ ἀφικόμενος πρῶτον ἐς Ἔφεσον θυσίαν ἐποιήσατο τῇ Ἀρτέμιδι.\n\n[4417] [ὅταν ὁ μετὰ τοῦτο τὸ θέρος χειμὼν τελευτήσῃ, ἓν καὶ εἰκοστὸν ἔτος πληροῦται.\n\n[4418] ]\n\n'

In [17]:
len(th1_series)

4678

In [18]:
len(el_series)

4678

In [19]:
th_df["thucydides_1863"] = th1_series

In [20]:
th_df

,Greek,thucydides_1863
0,,[0] Introduction.\n\n
1,,[1] L’auteur passe en revue les temps primitif...
2,,[2] i-xix.\n\n
3,,[3] But qu’il s’est proposé dans la rédaction ...
4,,[4] méthode et moyens qu’il a employés pour y ...
...,...,...
4673,"[4412] καὶ ὁ Τισσαφέρνης ἀπὸ τῆς Ἀσπένδου, ὡς ...",[10344] Lorsque Tissapherne apprit que la flot...
4674,[4413] ὄντων δὲ τῶν Πελοποννησίων ἐν τῷ Ἑλλησπ...,[10345] Cet Arsacès avait indignement traité c...
4675,[4414] φοβούμενοι οὖν αὐτὸν διὰ τοῦτο τὸ ἔργον...,[10346] Sous prétexte de combattre un ennemi q...
4676,[4415] ὁ δὲ Τισσαφέρνης αἰσθόμενος καὶ τοῦτο τ...,"[10347] puis, saisissant le moment de leur rep..."
